MODEL USED

In [6]:
MODEL = "llama3.2"

In [2]:
import pydantic
print(pydantic.__version__)


2.10.6


Initialization of model and Embeddings

In [7]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)
# model.invoke("where are you prividing your response from")


/var/folders/jm/g978mxy128vghxxjgmlf7cfc0000gn/T/ipykernel_7925/2696963359.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model=MODEL)
/var/folders/jm/g978mxy128vghxxjgmlf7cfc0000gn/T/ipykernel_7925/2696963359.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=MODEL)


Document loaded and splitted

In [9]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./temp_paper-image-captioning.pdf")
pages = loader.load_and_split()

In [11]:
# pages

In [4]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages= True
)

/var/folders/jm/g978mxy128vghxxjgmlf7cfc0000gn/T/ipykernel_7925/1801323123.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


creating prompt for LLM

In [14]:
from langchain.prompts import PromptTemplate

template = """
Answer the questions based on the context below. If you cannot answer the question given, just reply I don't know

Context: {context}
Chat History: {chat_history}
Question: {question}

"""

prompt = PromptTemplate.from_template(template)
# print(prompt.format(context="", question=""))

In [15]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

creating a chain 

In [16]:
chain = prompt | model | parser

In [20]:
response = chain.invoke(
    {
        "context":"nepal", 
        "chat_history":"I like pokhara very much",
        "question":"what is the city name"
    }
)

In [21]:
print(response)

Pokhara is a city in Nepal! So, to answer your question, the city name is Pokhara.


storing embeddings in vectorestores

In [22]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embeddings
)

/Users/nirajanpaudel17/anaconda3/envs/learn_crew/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


creating retreiver

In [23]:
retreiver = vectorstore.as_retriever()

In [24]:
retreiver.invoke("abstract")

[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2024-04-30T10:30:12+05:30', 'author': 'Microsoft account', 'moddate': '2024-04-30T10:30:12+05:30', 'source': './temp_paper-image-captioning.pdf', 'total_pages': 15, 'page': 11, 'page_label': '12'}, page_content="Nabaraj Subedi, Nirajan Paudel, Manish Chhetri, Sudarshan Acharya, Nabin Lamichhane  \n \nJournal of Soft Computing Paradigm, Month 2024, Volume 6, Issue 1  81 \n \n \n \n \n \n \n \n \n \nFigure 9. Sample Result 3 \n \n \n \n \n \n \n \nFigure 10. Sample Result 4 \nThe transformer model correctly predicted the object, environment, relations and the \nactions in the images with coherent sentences. \n Conclusion \nIn conclusion, our research work introduces a novel approach for generating detailed \nNepali paragraphs to describe images, leveraging both visual and linguistic struct ures. The \ntransformer's capacity to model long range dependencies on caption to effectively

In [25]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retreiver,
        "chat_history": lambda x: memory.load_memory_variables({})["chat_history"],
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

In [30]:
question = "who are the authors of this academic paper?"
response=chain.invoke({"question":question})
memory.save_context({"question": question}, {"answer": response})

In [31]:
print(response)

The authors of this academic paper are:

1. Nabaraj Subedi
2. Nirajan Paudel
3. Manish Chhetri
4. Sudarshan Acharya
5. Nabin Lamichhane


In [32]:
print(chain.invoke({"question": "what was my previous question?"}))

Your previous question was "who are the authors of this academic paper?"


for streaming responses

In [37]:
# to stream the answers 

for s in chain.stream({"question":"who are the authors of this academic paper?"}):
    print(s, end="", flush=True)

The authors of this academic paper are:

1. Nabaraj Subedi
2. Nirajan Paudel
3. Manish Chhetri
4. Sudarshan Acharya
5. Nabin Lamichhane